# Load Pnoi Sync data

In [5]:
import pandas as pd
import os
import numpy as np


mkdir = lambda p: 0 if os.path.exists(p) else (os.mkdir(p), 1)[1]

## Paths to Dataset
> paths valid for linux based os; make changes if running on windows

In [6]:
EXP_VER = "vC"

REPORTS = "reports"; mkdir(REPORTS) # Path to the reports folder

PNOI_CORPUS_CSV_NAME = "pnoicorpus_muster.csv" # csv file with meta data
PNOI_CORPUS_CSV_PATH = f"{REPORTS}/{PNOI_CORPUS_CSV_NAME}"  #"pnoicorpus_muster.csv" # Name of the csv file

PNOI_CORPUS_CSV_PATH = f"{REPORTS}/{PNOI_CORPUS_CSV_NAME}" # Path to the master csv file

PNOI_SYNC_CSV_NAME = f"pnoi_sync_aud_{EXP_VER}.csv" # Name of the csv file
PNOI_SYNC_CSV_PATH = f"{REPORTS}/{PNOI_SYNC_CSV_NAME}" # Path to the master csv file


In [7]:
class DataStaticInfo:
    VER = "*"
    SEP = "-"
    META_SEP = "_"
    EXT_SEP = "."
    ANOT_LABELS = ["aa", "ee", "uu", "oo", "ii", "xx", "bb1", "bb2", "bb3", "bb4"]

    fkeys = {
        "APP_CODE": "app_code",
        "SID":"sub_id",
        "FCLASS": "file_class",
        "FID": "file_ID",
        "COMNT": "file_comment",
        "FFMT": "file_format",
        "FNAME": "file_name",
        "FPATH": "file_path",
        "FMATCH": "file_match"
    }

class AudStaticData(DataStaticInfo):
    EMPTY_VAL = '-'
    FNAME_SEP = "-"
    ANOT_SEP = '\t'
    FS_k = "fs"
    DUR_k = "dur"
    BEGIN_k = "begin"; END_k = "end"; LABEL_k = "label"
    ANOTE_COLS = [BEGIN_k, END_k, LABEL_k]

    LUNG_LOCS = ["LU", "RU", "LL", "RL"]

    LBA_k = "LBA"; VBA_k = "VBA"; BA_k = "BA"
    
    AUD_TAG = "aud--"
    ANOT_TAG = "anot--"
    AUDIO_FPATH_k = f"audio--file_path"
    ANOT_FPATH_k = f"anot--file_path"


In [8]:
class AnotFrame(AudStaticData):

    PNOI_SYNC_DF: pd.DataFrame
    PNOI_SYNC_ANOT_DF: pd.DataFrame

    def __init__(self, sync_aud_csv_path: str) -> None:
        self.PNOI_SYNC_DF = pd.read_csv(sync_aud_csv_path)
        self.PNOI_SYNC_ANOT_DF = self.make_dataframe()

    def extract_anots(self, row: pd.Series) -> pd.DataFrame:
        anot_path = row[self.ANOT_FPATH_k]

        df = pd.read_csv(anot_path, sep=self.ANOT_SEP, names=self.ANOTE_COLS)
        df[self.DUR_k] = df[self.END_k] - df[self.BEGIN_k]

        info = pd.DataFrame([row]* len(df)).reset_index(drop=True)
        return pd.concat([df, info], axis=1)


    def make_dataframe(self) -> pd.DataFrame:
        
        anot_dfs = []
        for _, row in self.PNOI_SYNC_DF.iterrows():
            anot_df = self.extract_anots(row)
            anot_dfs.append(anot_df)

        return pd.concat(anot_dfs, axis=0).reset_index(drop=True)

pnoidata_anotframe = AnotFrame(PNOI_SYNC_CSV_PATH)

pnoidata_anotframe.PNOI_SYNC_ANOT_DF.head()

,begin,end,label,dur,fs,sub_id,file_class,audio--file_path,anot--file_path
0,0.200000,1.723096,oo,1.523096,16000,shreyamgupta_78aa423a,_before_LU,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...
1,3.722159,5.007272,ii,1.285113,16000,shreyamgupta_78aa423a,_before_LU,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...
2,5.007272,5.959207,xx,0.951935,16000,shreyamgupta_78aa423a,_before_LU,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...
3,5.959207,6.911141,ii,0.951934,16000,shreyamgupta_78aa423a,_before_LU,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...
4,6.911141,8.101060,xx,1.189919,16000,shreyamgupta_78aa423a,_before_LU,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...,reports/pnoi_sync_data_vC/shreyamgupta_78aa423...


In [9]:
class PnoiAnotDataset:

    PNOI_SYNC_ANOT_DF: pd.DataFrame
    PNOI_MUSTER_DF: pd.DataFrame
    PNOI_ANOT_DATASET_DF: pd.DataFrame

    def __init__(self, anot_df: pd.DataFrame, pnoi_muster_csv) -> None:
        self.PNOI_SYNC_ANOT_DF = anot_df.copy()
        self.PNOI_MUSTER_DF = pd.read_csv(pnoi_muster_csv)
 
        self.PNOI_ANOT_DATASET_DF = self.make_dataframe()
        

    col_order = [
    'sub_id',

    'subjectName', 'subjectGender', 'subjectAge',
    'subjectType', 'subjectHeight', 'subjectWeight',
    
    'anot--META--file_path',

    'FEV1_ref_before', 'FEV1_val_before',
    'FVC_ref_before', 'FVC_val_before',
    'ratio_ref_before', 'ratio_val_before',
    
    'FEV1_ref_after', 'FEV1_val_after',
    'FVC_ref_after', 'FVC_val_after',
    'ratio_ref_after', 'ratio_val_after',
    ]

    def make_dataframe(self) -> pd.DataFrame:
        pnoi_anot_dataset = self.PNOI_SYNC_ANOT_DF.merge(self.PNOI_MUSTER_DF[self.col_order], on="sub_id", how="left")

        return pnoi_anot_dataset

    

pnoidata_anotdataset = PnoiAnotDataset(pnoidata_anotframe.PNOI_SYNC_ANOT_DF, PNOI_CORPUS_CSV_PATH)
cols_filt = pnoidata_anotdataset.PNOI_MUSTER_DF.columns.str.contains("after")
cols = pnoidata_anotdataset.PNOI_MUSTER_DF.columns[~cols_filt]

pnoidata_anotdataset.PNOI_MUSTER_DF[cols].replace("-", np.nan).dropna().apply(pd.to_numeric, errors='ignore')['subjectType'].value_counts()



subjectType
Patient    11
Name: count, dtype: int64

In [10]:
pnoidata_anotdataset.PNOI_ANOT_DATASET_DF.to_csv(f"{REPORTS}/pnoi_anot_dataset_test{EXP_VER}.csv", index=False)

### Test Data Count per Subject

In [11]:
subjects = pnoidata_anotframe.PNOI_SYNC_DF["sub_id"].unique()

for sub in subjects:
    filt = pnoidata_anotframe.PNOI_SYNC_DF["sub_id"] == sub
    sub_df = pnoidata_anotframe.PNOI_SYNC_DF[filt]

    if len(sub_df)%4 != 0: print(len(sub_df), sub) # there should be 4 or 8 audio files per subject

7 sujatan_bdd161b6


In [14]:
pnoidata_anotframe.PNOI_SYNC_ANOT_DF["label"].value_counts()

label
ii     736
xx     736
ee     160
uu     160
oo     158
aa     158
bb1     18
bb2     18
bb4     18
bb3     17
Name: count, dtype: int64